<a href="https://colab.research.google.com/github/martinbedia/mineria/blob/main/ejercicio%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1
Sobre el código realizado durante las clases para analizar tickets estudia si los tickets
contienen parking o no. Aquellos que lo contienen aportan información adicional. Modifica el programa para tener esto en cuenta. El resultado del código deberán ser dos archivos csv. Uno tendrá las columnas id ticket, producto, cantidad, precio por unidad. El otro tendrá para cada id de ticket, parking (booleana), fecha, total precio ticket, hora entrada, hora salida, tiempo total (que podéis calcularlo de las anteriores). Las columnas para las que no se dispone de datos en algunos de los tickets deberán ser NA.

In [1]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.8 MB/s eta 0:00:00


In [2]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import KMeans, AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os
import PyPDF2
import re
import csv
from datetime import datetime

## Funciones individuales
Para realizar el ejercicio, realizare funciones individuales de cada operacion que me manda para en el siguiente paso, unir lo relevante en una unica funcion que me devuelva el archivo csv. De esta manera veo que me va saliendo el ejercicio y voy obteniendo los resultados que busco.

In [3]:
# leer archivos de un tipo concreto de una carpeta y sus subcarpetas
# recursive para que busque en las subcarpetas
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

['./20230921 Mercadona 9,70 €.pdf', './20230909 Mercadona 21,23 €.pdf']

Veo que me salen los tickets sobre los que trabajaremos.

In [21]:
# leer el ticket
pdf_reader = PyPDF2.PdfReader(tickets[0])    # conectar el archivo pdf con el codigo en python
ticket = pdf_reader.pages[0]   # acceder a la primera pagina
texto_ticket = ticket.extract_text()
texto_ticket

'MERCADONA, S.A.   A-46103834\nC/ ALFONSO ARGUELLES 2\n33900 LANGREO\nTELÉFONO: 985698053\n21/09/2023 17:12  OP: 371122\nFACTURA SIMPLIFICADA: 2363-011-208468\nDescripción P. Unit Importe\n1MÁSCARA DE CEJAS 4,00\n1PICATOSTE AJO 0,85\n1TAQUITOS SALMÓN 3,80\n1DUO CANONIGOS RUCULA 1,05\nTOTAL (€) 9,70\nTARJETA BANCARIA 9,70\nIVA BASE IMPONIBLE (€) CUOTA (€)\n10% 4,23 0,42\n21% 3,31 0,69\n0% 1,05 0,00\nTOTAL 8,59 1,11\nTARJ. BANCARIA:  **** **** **** 4016\nN.C: 003765716  AUT: 766364\nAID: A0000000041010   ARC: 3030\nMASTERCARD\nImporte: 9,70 €  MASTERCARD\nSE ADMITEN DEVOLUCIONES CON TICKET'

In [22]:
# funcion para leer todos los tickets a la vez, aunque trabajaremos con la anterior que los hace de uno en uno
def extraer_texto(tickets):
    # Listas para almacenar la información de cada ticket
    texto_ticket = []

    for ticket_path in tickets:
        # Conectar el archivo PDF con el código en Python
        pdf_reader = PyPDF2.PdfReader(ticket_path)
        # Acceder a la primera página
        ticket = pdf_reader.pages[0]
        # Extraer texto del ticket
        texto = ticket.extract_text()
        # Agregar el texto del ticket a la lista
        texto_ticket.append(texto)
    return texto_ticket

textos_tickets = extraer_texto(tickets)


In [23]:
# texto del ticket por lineas
lineas = np.array(texto_ticket.split('\n'))
lineas

array(['MERCADONA, S.A.   A-46103834', 'C/ ALFONSO ARGUELLES 2',
       '33900 LANGREO', 'TELÉFONO: 985698053',
       '21/09/2023 17:12  OP: 371122',
       'FACTURA SIMPLIFICADA: 2363-011-208468',
       'Descripción P. Unit Importe', '1MÁSCARA DE CEJAS 4,00',
       '1PICATOSTE AJO 0,85', '1TAQUITOS SALMÓN 3,80',
       '1DUO CANONIGOS RUCULA 1,05', 'TOTAL (€) 9,70',
       'TARJETA BANCARIA 9,70', 'IVA BASE IMPONIBLE (€) CUOTA (€)',
       '10% 4,23 0,42', '21% 3,31 0,69', '0% 1,05 0,00',
       'TOTAL 8,59 1,11', 'TARJ. BANCARIA:  **** **** **** 4016',
       'N.C: 003765716  AUT: 766364', 'AID: A0000000041010   ARC: 3030',
       'MASTERCARD', 'Importe: 9,70 €  MASTERCARD',
       'SE ADMITEN DEVOLUCIONES CON TICKET'], dtype='<U37')

Meto lo anterior en una funcion a la que le diga la ruta y me devuelva el array con las lineas



In [24]:
# lineas_ticekt1 = leer_lineas_de_pdf(archivo)

def leer_lineas_de_pdf (nombre_del_pdf):
  pdf_reader = PyPDF2.PdfReader(nombre_del_pdf)
  ticket = pdf_reader.pages[0]
  texto_ticket = ticket.extract_text()
  lineas = np.array(texto_ticket.split('\n'))
  return lineas

In [25]:
lineas_ticekt1 = leer_lineas_de_pdf('/content/20230909 Mercadona 21,23 €.pdf')
lineas_ticekt1

array(['MERCADONA, S.A.   A-46103834', 'PLAZA DAOIZ Y VELARDE 5',
       '33009 OVIEDO', 'TELÉFONO: 985216153',
       '09/09/2023 20:35  OP: 191484',
       'FACTURA SIMPLIFICADA: 3433-011-051093',
       'Descripción P. Unit Importe', '2TAQUITOS SALMÓN 3,80 7,60',
       '1TROTTOLE TRICOLORE 0,95', '1QUESO RALLADO GOUDA 1,44',
       '1ÑOQUIS DE PATATA 1,12', '2LECHE SEMI S/LACT 1,46 2,92',
       '1QUESO FETA 2,35', '1PICATOSTE AJO 0,85',
       '2NEGRA RODAJAS P3 2,00 4,00', '1PARKING 0,00',
       'ENTRADA  20:22       SALIDA  20:35', 'TOTAL (€) 21,23',
       'TARJETA BANCARIA 21,23', 'IVA BASE IMPONIBLE (€) CUOTA (€)',
       '10% 11,32 1,13', '5% 1,97 0,10', '0% 6,71 0,00',
       'TOTAL 20,00 1,23', 'TARJ. BANCARIA:  **** **** **** 4016',
       'N.C: 003943818  AUT: 729144', 'AID: A0000000041010   ARC: 3030',
       'MASTERCARD', 'Importe: 21,23 € MASTERCARD',
       'SE ADMITEN DEVOLUCIONES CON TICKET', 'DISPONE DE 20 MINUTOS',
       'PARA RETIRAR SU VEHÍCULO'], dtype='<U37

Ahora, una vez que tengo la informacion, debo extraer el id del ticket, los productos, la cantidad y el precio por unidad. Usare expresiones regulares para identificar esa informacion en el texto en bruto.

In [26]:
def procesar_ticket_lineas(lineas):
    id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
    id_ticket = id_ticket_match.group() if id_ticket_match else None

    productos = []
    for linea in lineas:
        producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())

    return id_ticket, productos

In [27]:
ticket1_prepro = procesar_ticket_lineas(lineas_ticekt1)
ticket1_pro = ticket1_prepro[1]
ticket1_pro

<ipython-input-26-21d72240d8d7>:7: FutureWarning: Possible nested set at position 20
  producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)


['2TAQUITOS SALMÓN 3,80 7,60',
 '1TROTTOLE TRICOLORE 0,95',
 '1QUESO RALLADO GOUDA 1,44',
 '1ÑOQUIS DE PATATA 1,12',
 '2LECHE SEMI S/LACT 1,46 2,92',
 '1QUESO FETA 2,35',
 '1PICATOSTE AJO 0,85',
 '2NEGRA RODAJAS P3 2,00 4,00',
 '1PARKING 0,00']

In [28]:
id_ticket = ticket1_prepro[0]
id_ticket

'3433-011-051093'

In [29]:
def obtener_cantidad(linea):
    # Patrón para extraer el número inicial de la línea
    patron = r'^(\d{1,})'

    # Buscar coincidencias en la línea
    coincidencia = re.search(patron, linea)

    if coincidencia:
        return coincidencia.group()
    else:
        return None

# Obtener las cantidades
cantidades = [obtener_cantidad(linea) for linea in ticket1_pro]

# Imprimir las cantidades
print("Cantidades:", cantidades)


Cantidades: ['2', '1', '1', '1', '2', '1', '1', '2', '1']


In [30]:
def obtener_nombre_producto(linea):
    # Patrón para extraer el nombre del producto
    patron = r'^\d{1,}([A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*\s)'

    # Buscar coincidencias en la línea
    coincidencia = re.search(patron, linea, flags=re.IGNORECASE)

    if coincidencia:
        return coincidencia.group(1).strip()
    else:
        return None

# Obtener los nombres de los productos
nombres_productos = [obtener_nombre_producto(linea) for linea in ticket1_pro]

# Imprimir los nombres de los productos
print("Nombres de Productos:", nombres_productos)

Nombres de Productos: ['TAQUITOS SALMÓN', 'TROTTOLE TRICOLORE', 'QUESO RALLADO GOUDA', 'ÑOQUIS DE PATATA', 'LECHE SEMI S/LACT', 'QUESO FETA', 'PICATOSTE AJO', 'NEGRA RODAJAS P3', 'PARKING']


In [31]:
def obtener_precio_producto(linea):
    # Patrón para extraer el precio del producto
    patron = r'(\d{1,2},\d{1,2})'

    # Buscar coincidencias en la línea
    coincidencia = re.search(patron, linea)

    if coincidencia:
        return coincidencia.group(1)
    else:
        return None

# Obtener los precios de los productos
precios_productos = [obtener_precio_producto(linea) for linea in ticket1_pro]

# Imprimir los precios de los productos
print("Precios de Productos:", precios_productos)

Precios de Productos: ['3,80', '0,95', '1,44', '1,12', '1,46', '2,35', '0,85', '2,00', '0,00']


In [32]:
df = pd.DataFrame({
    'id ticket': id_ticket,
    'cantidad': cantidades,
    'producto': nombres_productos,
    'precio_individual': precios_productos
})
df

,id ticket,cantidad,producto,precio_individual
0,3433-011-051093,2,TAQUITOS SALMÓN,"3,80"
1,3433-011-051093,1,TROTTOLE TRICOLORE,"0,95"
2,3433-011-051093,1,QUESO RALLADO GOUDA,"1,44"
3,3433-011-051093,1,ÑOQUIS DE PATATA,"1,12"
4,3433-011-051093,2,LECHE SEMI S/LACT,"1,46"
5,3433-011-051093,1,QUESO FETA,"2,35"
6,3433-011-051093,1,PICATOSTE AJO,"0,85"
7,3433-011-051093,2,NEGRA RODAJAS P3,"2,00"
8,3433-011-051093,1,PARKING,"0,00"


Ahora haré la segunda parte del ejercicio, donde ver para cada ticket su id, parking (booleana), fecha, total precio ticket, hora entrada, hora salida, tiempo total. Las columnas para las que no se dispone de datos en algunos de los tickets deberán ser NA.

In [33]:
def parking_TOF(lineas):
    # Buscar la presencia de "1PARKING" en el ticket
    parking_match = re.search(r'1PARKING', '\n'.join(lineas), flags=re.IGNORECASE)

    # Determinar si el ticket contiene "1PARKING"
    contiene_parking = True if parking_match else False

    return contiene_parking

parking = parking_TOF(lineas_ticekt1)
parking

True

In [34]:
def obtener_horas_parking(lineas):
    # Patrón para extraer las horas de entrada y salida del parking
    patron_horas_parking = r'ENTRADA\s+(\d{2}:\d{2})\s+SALIDA\s+(\d{2}:\d{2})'

    # Buscar coincidencias en las líneas del ticket
    coincidencia_horas_parking = re.search(patron_horas_parking, '\n'.join(lineas), flags=re.IGNORECASE)

    # Obtener las horas de entrada y salida si hay coincidencia
    horas_entrada, horas_salida = coincidencia_horas_parking.groups() if coincidencia_horas_parking else (None, None)

    return horas_entrada, horas_salida


horas_entrada, horas_salida = obtener_horas_parking(lineas_ticekt1)

print("Hora de Entrada:", horas_entrada)
print("Hora de Salida:", horas_salida)

Hora de Entrada: 20:22
Hora de Salida: 20:35


In [35]:
from datetime import datetime

def calcular_tiempo_total(hora_entrada, hora_salida):
    # Convertir las cadenas de hora a objetos de tipo datetime
    hora_entrada_obj = datetime.strptime(hora_entrada, '%H:%M')
    hora_salida_obj = datetime.strptime(hora_salida, '%H:%M')

    # Calcular la diferencia de tiempo
    tiempo_total = hora_salida_obj - hora_entrada_obj

    # Extraer la duración en horas y minutos
    horas, minutos = divmod(tiempo_total.seconds // 60, 60)

    return f"{horas} horas y {minutos} minutos"

In [36]:
def procesar_parking(lineas):
    # Patrón para extraer las horas de entrada y salida del parking
    patron_horas_parking = r'ENTRADA\s+(\d{2}:\d{2})\s+SALIDA\s+(\d{2}:\d{2})'
    # entendemos que por ejemplo las 9 am en el ticket vendria como 09 y no como tan solo 9

    # Buscar coincidencias en las líneas del ticket
    coincidencia_horas_parking = re.search(patron_horas_parking, '\n'.join(lineas), flags=re.IGNORECASE)

    # Obtener las horas de entrada y salida si hay coincidencia
    horas_entrada, horas_salida = coincidencia_horas_parking.groups() if coincidencia_horas_parking else (None, None)

    # Calcular el tiempo total en el supermercado
    tiempo_total = None
    if horas_entrada and horas_salida:
        tiempo_total = calcular_tiempo_total(horas_entrada, horas_salida)

    return horas_entrada, horas_salida, tiempo_total


# Prueba de la función
horas_entrada, horas_salida, tiempo_total = procesar_parking(lineas_ticekt1)

print("Hora de Entrada:", horas_entrada)
print("Hora de Salida:", horas_salida)
print("Tiempo Total en el Supermercado:", tiempo_total)


Hora de Entrada: 20:22
Hora de Salida: 20:35
Tiempo Total en el Supermercado: 0 horas y 13 minutos


In [39]:
def procesar_precio(lineas):
    precio_match = re.search(r'TOTAL.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
    precio_total = precio_match.group() if precio_match else None

    total_match = re.search(r'\d{1,2},\d{1,2}', precio_total)
    total_pagado = total_match.group() if total_match else None

    return total_pagado

total_pagado = procesar_precio(lineas_ticekt1)
total_pagado

'21,23'

In [40]:
def extraer_fecha(lineas):
    # Patrón para buscar fechas en formato DD/MM/YYYY
    fecha_match = re.search(r'\b(\d{2}/\d{2}/\d{4})\b', '\n'.join(lineas))
    fecha_final = fecha_match.group() if fecha_match else None

    return fecha_final

fecha_1 = extraer_fecha(lineas_ticekt1)
fecha_1

'09/09/2023'

In [41]:
df = pd.DataFrame({
    'id ticket': id_ticket,
    'parking': parking,
    'fecha': fecha_1,
    'total precio': total_pagado,
    'hora entrada': horas_entrada,
    'hora salida': horas_salida,
    'tiempo total': tiempo_total,
},index=[0])
df

,id ticket,parking,fecha,total precio,hora entrada,hora salida,tiempo total
0,3433-011-051093,True,09/09/2023,"21,23",20:22,20:35,0 horas y 13 minutos


## Función final
En este apartado, tomando la información del paso previo, haré una única funcion que me devuelva el resultado que me pide el ejercicio que es un csv con el id ticket, producto, cantidad y precio por unidad. También haré otra que me devuelva el id e ticket, parking (booleana), fecha, total precio ticket, hora entrada, hora salida y tiempo total.

In [65]:
# todo en la misma funcion
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

def ejercicio1(tickets):
  # leer el ticket
  pdf_reader = PyPDF2.PdfReader(tickets)    # conectar el archivo pdf con el codigo en python
  ticket = pdf_reader.pages[0]   # acceder a la primera pagina
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  productos = []
  for linea in lineas:
        producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())


  # Buscar coincidencias en la cantidad
  cantidades = []
  for linea in productos:
    cantidad_match = re.search(r'^(\d{1,})',linea)
    cantidades.append(cantidad_match.group(1) if cantidad_match else None)

  # Buscar coincidencias en el nombre
  nombres = []
  for linea in productos:
    nombres_matches = re.search(r'^\d{1,}([A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*\s)', linea, flags=re.IGNORECASE)
    nombres.append(nombres_matches.group(1).strip() if nombres_matches else None)

  # Buscar coincidencias en el precio
  precios = []
  for linea in productos:
    precio_matches = re.search(r'(\d{1,2},\d{1,2})', linea)
    precios.append(precio_matches.group(1) if precio_matches else None)

  df = pd.DataFrame({
    'id ticket': id_ticket,
    'cantidad': cantidades,
    'producto': nombres,
    'precio_individual': precios
  })


  return df

In [67]:
ejercicio1('/content/20230909 Mercadona 21,23 €.pdf')

,id ticket,cantidad,producto,precio_individual
0,3433-011-051093,2,TAQUITOS SALMÓN,"3,80"
1,3433-011-051093,1,TROTTOLE TRICOLORE,"0,95"
2,3433-011-051093,1,QUESO RALLADO GOUDA,"1,44"
3,3433-011-051093,1,ÑOQUIS DE PATATA,"1,12"
4,3433-011-051093,2,LECHE SEMI S/LACT,"1,46"
5,3433-011-051093,1,QUESO FETA,"2,35"
6,3433-011-051093,1,PICATOSTE AJO,"0,85"
7,3433-011-051093,2,NEGRA RODAJAS P3,"2,00"
8,3433-011-051093,1,PARKING,"0,00"


Ahora haremos la del parking

In [44]:
# todo en la misma funcion
ruta_carpeta_raiz = '.'
tickets = glob.glob(os.path.join(ruta_carpeta_raiz, '**/*.pdf'),recursive=True)
tickets

def ejercicio1_parking(tickets):
  # leer el ticket
  pdf_reader = PyPDF2.PdfReader(tickets)    # conectar el archivo pdf con el codigo en python
  ticket = pdf_reader.pages[0]   # acceder a la primera pagina
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None


  # Buscar la presencia de "1PARKING" en el ticket
  parking_match = re.search(r'1PARKING', '\n'.join(lineas), flags=re.IGNORECASE)
  # Determinar si el ticket contiene "1PARKING"
  contiene_parking = True if parking_match else False

  fecha_match = re.search(r'\b(\d{2}/\d{2}/\d{4})\b', '\n'.join(lineas))
  fecha_final = fecha_match.group() if fecha_match else None

  precio_match = re.search(r'TOTAL.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  precio_total = precio_match.group() if precio_match else None
  total_match = re.search(r'\d{1,2},\d{1,2}', precio_total)
  total_pagado = total_match.group() if total_match else None

  # Patrón para extraer las horas de entrada y salida del parking
  patron_horas_parking = r'ENTRADA\s+(\d{2}:\d{2})\s+SALIDA\s+(\d{2}:\d{2})'
  # Buscar coincidencias en las líneas del ticket
  coincidencia_horas_parking = re.search(patron_horas_parking, '\n'.join(lineas), flags=re.IGNORECASE)
  # Obtener las horas de entrada y salida si hay coincidencia
  horas_entrada, horas_salida = coincidencia_horas_parking.groups() if coincidencia_horas_parking else (None, None)

  if contiene_parking == True:
    hora_entrada_obj = datetime.strptime(horas_entrada, '%H:%M')
    hora_salida_obj = datetime.strptime(horas_salida, '%H:%M')
    # Calcular la diferencia de tiempo
    tiempo_total = None
    if horas_entrada and horas_salida:
        tiempo_total = hora_salida_obj - hora_entrada_obj
        # Extraer la duración en horas y minutos
        horas, minutos = divmod(tiempo_total.seconds // 60, 60)
        tiempo_total = f"{horas} horas y {minutos} minutos"
  else: tiempo_total = None



  df = pd.DataFrame({
    'id ticket': id_ticket,
    'parking': contiene_parking,
    'fecha': fecha_final,
    'total precio': total_pagado,
    'hora entrada': horas_entrada,
    'hora salida': horas_salida,
    'tiempo total': tiempo_total,
  },index=[0])

  return df



In [45]:
ejercicio1_parking('/content/20230921 Mercadona 9,70 €.pdf')

,id ticket,parking,fecha,total precio,hora entrada,hora salida,tiempo total
0,2363-011-208468,False,21/09/2023,"9,70",None,None,None


Finalmente, haremos una funcion conjunta si no queremos tener la informacion por separado.

In [68]:
def ejercicio1_final(tickets):
  # leer el ticket
  pdf_reader = PyPDF2.PdfReader(tickets)    # conectar el archivo pdf con el codigo en python
  ticket = pdf_reader.pages[0]   # acceder a la primera pagina
  texto_ticket = ticket.extract_text()
  # texto del ticket por lineas
  lineas = np.array(texto_ticket.split('\n'))

  id_ticket_match = re.search(r'\d*-\d*-\d*', '\n'.join(lineas))
  id_ticket = id_ticket_match.group() if id_ticket_match else None

  productos = []
  for linea in lineas:
        producto_match = re.search(r'\d[A-ZÑñÁÉÍÓÚáéíóú][[A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*((\d{1,2},\d{1,2}\s*){1,2})', linea, flags=re.IGNORECASE)
        if producto_match:
            productos.append(producto_match.group())


  # Buscar coincidencias en la cantidad
  cantidades = []
  for linea in productos:
    cantidad_match = re.search(r'^(\d{1,})',linea)
    cantidades.append(cantidad_match.group(1) if cantidad_match else None)

  # Buscar coincidencias en el nombre
  nombres = []
  for linea in productos:
    nombres_matches = re.search(r'^\d{1,}([A-ZÑñÁÉÍÓÚáéíóú/0-9\s]*\s)', linea, flags=re.IGNORECASE)
    nombres.append(nombres_matches.group(1).strip() if nombres_matches else None)

  # Buscar coincidencias en el precio
  precios = []
  for linea in productos:
    precio_matches = re.search(r'(\d{1,2},\d{1,2})', linea)
    precios.append(precio_matches.group(1) if precio_matches else None)


  # Buscar la presencia de "1PARKING" en el ticket
  parking_match = re.search(r'1PARKING', '\n'.join(lineas), flags=re.IGNORECASE)
  # Determinar si el ticket contiene "1PARKING"
  contiene_parking = True if parking_match else False

  fecha_match = re.search(r'\b(\d{2}/\d{2}/\d{4})\b', '\n'.join(lineas))
  fecha_final = fecha_match.group() if fecha_match else None

  precio_match = re.search(r'TOTAL.*\s\d{1,2},\d{1,2}', '\n'.join(lineas))
  precio_total = precio_match.group() if precio_match else None
  total_match = re.search(r'\d{1,2},\d{1,2}', precio_total)
  total_pagado = total_match.group() if total_match else None

  # Patrón para extraer las horas de entrada y salida del parking
  patron_horas_parking = r'ENTRADA\s+(\d{2}:\d{2})\s+SALIDA\s+(\d{2}:\d{2})'
  # Buscar coincidencias en las líneas del ticket
  coincidencia_horas_parking = re.search(patron_horas_parking, '\n'.join(lineas), flags=re.IGNORECASE)
  # Obtener las horas de entrada y salida si hay coincidencia
  horas_entrada, horas_salida = coincidencia_horas_parking.groups() if coincidencia_horas_parking else (None, None)

  if contiene_parking == True:
    hora_entrada_obj = datetime.strptime(horas_entrada, '%H:%M')
    hora_salida_obj = datetime.strptime(horas_salida, '%H:%M')
    # Calcular la diferencia de tiempo
    tiempo_total = None
    if horas_entrada and horas_salida:
        tiempo_total = hora_salida_obj - hora_entrada_obj
        # Extraer la duración en horas y minutos
        horas, minutos = divmod(tiempo_total.seconds // 60, 60)
        tiempo_total = f"{horas} horas y {minutos} minutos"
  else: tiempo_total = None


  df1 = pd.DataFrame({
    'id ticket': id_ticket,
    'cantidad': cantidades,
    'producto': nombres,
    'precio_individual': precios
  })

  df2 = pd.DataFrame({
    'id ticket': id_ticket,
    'parking': contiene_parking,
    'fecha': fecha_final,
    'total precio': total_pagado,
    'hora entrada': horas_entrada,
    'hora salida': horas_salida,
    'tiempo total': tiempo_total,
  },index=[0])

  return df1, df2

In [69]:
ejercicio1_final('/content/20230921 Mercadona 9,70 €.pdf')

(         id ticket cantidad              producto precio_individual
 0  2363-011-208468        1      MÁSCARA DE CEJAS              4,00
 1  2363-011-208468        1         PICATOSTE AJO              0,85
 2  2363-011-208468        1       TAQUITOS SALMÓN              3,80
 3  2363-011-208468        1  DUO CANONIGOS RUCULA              1,05,
          id ticket  parking       fecha total precio hora entrada hora salida  \
 0  2363-011-208468    False  21/09/2023         9,70         None        None   
 
   tiempo total  
 0         None  )

In [70]:
ejercicio1_final('/content/20230909 Mercadona 21,23 €.pdf')

(         id ticket cantidad             producto precio_individual
 0  3433-011-051093        2      TAQUITOS SALMÓN              3,80
 1  3433-011-051093        1   TROTTOLE TRICOLORE              0,95
 2  3433-011-051093        1  QUESO RALLADO GOUDA              1,44
 3  3433-011-051093        1     ÑOQUIS DE PATATA              1,12
 4  3433-011-051093        2    LECHE SEMI S/LACT              1,46
 5  3433-011-051093        1           QUESO FETA              2,35
 6  3433-011-051093        1        PICATOSTE AJO              0,85
 7  3433-011-051093        2     NEGRA RODAJAS P3              2,00
 8  3433-011-051093        1              PARKING              0,00,
          id ticket  parking       fecha total precio hora entrada hora salida  \
 0  3433-011-051093     True  09/09/2023        21,23        20:22       20:35   
 
            tiempo total  
 0  0 horas y 13 minutos  )